In [11]:
from sys import path
from pathlib import Path
from sqlalchemy import create_engine

import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format

db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

engine = create_engine(f'sqlite:///{db_file_path_str}')

In [13]:
with engine.connect() as conn, conn.begin():  
    ventas_enero = pd.read_sql_table('ventas_enero', conn, dtype_backend='numpy_nullable')
    ventas_febrero = pd.read_sql_table('ventas_febrero', conn, dtype_backend='numpy_nullable')
    ventas_marzo = pd.read_sql_table('ventas_marzo', conn, dtype_backend='numpy_nullable')
    ventas_abril = pd.read_sql_table('ventas_abril', conn, dtype_backend='numpy_nullable')
    ventas_mayo = pd.read_sql_table('ventas_mayo', conn, dtype_backend='numpy_nullable')
    ventas_año = pd.read_sql_table('ventas_año', conn, dtype_backend='numpy_nullable')

engine.dispose()

In [23]:
ventas = ventas_enero
vend_id = 227

vend_nom = ventas[ventas['salesperson_id'] == vend_id].iloc[0,15]

ventas[ventas['salesperson_id'] == vend_id].head()

,fact_doc_id,name,invoice_date,state,invoice_origin,module_origin,pos_doc_id,move_type,reversal_move_id,reversed_entry_id,...,warehouse,fact_line_id,product_id,product_name,prod_código,prod_línea,quantity,price_unit,discount,price_subtotal
11,130,F2-CC/2024/00002,2024-01-02,posted,PdV SJC/0010,PdV,23,out_invoice,<NA>,<NA>,...,A2,361,8764,"Dis.Metal MK D-71685 (D-18409) 4.5"" *11465*",11465,0MAKA,9.00,21.13,0.00,190.17
26,158,F2-CC/2024/00008,2024-01-02,posted,PdV SJC/0018,PdV,37,out_invoice,<NA>,<NA>,...,A2,462,11462,"Cincel Corte Frio 1x8"" #12160 Truper *121600*",121600,00TRU,1.00,124.82,0.00,124.82
27,160,F2-CC/2024/00009,2024-01-02,posted,PdV SJC/0019,PdV,38,out_invoice,<NA>,<NA>,...,A2,469,9113,Cepillo D/Alamb C/M Trup CEA-10 C #11540 *9585*,9585,00TRU,1.00,28.36,0.00,28.36
30,164,F2-CC/2024/00011,2024-01-02,posted,PdV SJC/0021,PdV,43,out_invoice,<NA>,<NA>,...,A2,482,9350,Cincho Volteck 35cm Negro #44329 (25pz) *13635*,13635,00TRU,10.00,34.04,0.00,340.40
31,164,F2-CC/2024/00011,2024-01-02,posted,PdV SJC/0021,PdV,43,out_invoice,<NA>,<NA>,...,A2,483,9077,Cincho Volteck 20cm Negro #44324 (50Pz) *18648*,18648,00TRU,10.00,27.74,0.00,277.40


¿Cuál es la órden de venta dónde más vendió y cuánto?

In [ ]:
wep = vendedora_df[['invoice_origin','price_subtotal']].groupby('invoice_origin').sum().sort_values(by=['price_subtotal'],ascending=False).iloc[0]

f"{vend_nom} vendió ${wep['price_subtotal']:,.2f} en la orden {wep.name}"

Dame la orden de venta mas grande por vendedora

In [ ]:
for id in descrip_sales_users_df['id']:
    vend_id_for = id
    vend_nom_for = descrip_sales_users_df.loc[descrip_sales_users_df['id'] == vend_id_for]['name'].iloc[0]

    vendedora_df_for = ventas[ventas['salesperson_id'] == vend_id_for]
    if not vendedora_df_for[['invoice_origin','price_subtotal']].groupby('invoice_origin').sum().sort_values(by=['price_subtotal'],ascending=False).empty:
        wep = vendedora_df_for[['invoice_origin','price_subtotal']].groupby('invoice_origin').sum().sort_values(by=['price_subtotal'],ascending=False).iloc[0]
        print(f"{vend_nom_for} vendió ${wep['price_subtotal']:,.2f} en la orden {wep.name}")

¿Cuáles son los 5 artículos que más vende? Por Cantidad

In [ ]:
wep = vendedora_df[['product_id','quantity']].groupby('product_id').sum().sort_values(by=['quantity'],ascending=False).iloc[0:5]
print(f"{vend_nom}")

¿Cuáles son los 5 artículos que más vende? Por precio

In [ ]:
wep = vendedora_df[['product_id','price_subtotal']].groupby('product_id').sum().sort_values(by=['price_subtotal'],ascending=False).iloc[0:5]
print(f"{vend_nom}")

¿Cuáles son los mejores 3 clientes de la vendedora? 1ro por cantidad de productos vendidos, 2do por subtotal de venta y el último por cantidad de OV en el mes.

In [ ]:
wep = vendedora_df[['partner_id','quantity','price_subtotal']].groupby('partner_id').agg({'quantity':['sum',],'price_subtotal':['sum',]})

cprod = wep[('quantity', 'sum')].sort_values(ascending=False)
cprod_name = cprod.keys()[0]
cprod_cant = cprod.loc[cprod_name]
csub = wep[('price_subtotal', 'sum')].sort_values(ascending=False)
csub_name = csub.keys()[0]
csub_cant = csub.loc[csub_name]


wep = vendedora_df[['partner_id','invoice_origin']].groupby(['partner_id', 'invoice_origin']).count().reset_index().groupby('partner_id').count()

cov = wep['invoice_origin'].sort_values(ascending=False)
cov_name = cov.keys()[0]
cov_cant = cov.loc[cov_name]


In [ ]:
print(f'La vendedora {vend_nom} tiene a sus 3 mejores clientes de la siguiente manera:')
print(f'\tEl cliente {cov_name} es el cliente que más órdenes de venta tiene, siendo un total de {cov_cant:,.2f} órdenes')
print(f'\tEl cliente {cprod_name} es el cliente que más cantidad de producto, siendo un total de {cprod_cant:,.2f} artículos vendidos')
print(f'\tEl cliente {csub_name} es el cliente que más facturó, siendo un total de ${csub_cant:,.2f} pesos en el mes de enero')

Ahora resuelve de nuevo la pregunta anterir, sin tomar al cliente MOSTRADOR

In [ ]:
wep = vendedora_df.loc[vendedora_df['partner_id'] != 16253, ['partner_id','quantity','price_subtotal']].groupby('partner_id').agg({'quantity':['sum',],'price_subtotal':['sum',]})

cprod = wep[('quantity', 'sum')].sort_values(ascending=False)
cprod_name = cprod.keys()[0]
cprod_cant = cprod.loc[cprod_name]
csub = wep[('price_subtotal', 'sum')].sort_values(ascending=False)
csub_name = csub.keys()[0]
csub_cant = csub.loc[csub_name]


wep = vendedora_df.loc[vendedora_df['partner_id'] != 16253, ['partner_id','invoice_origin']].groupby(['partner_id', 'invoice_origin']).count().reset_index().groupby('partner_id').count()

cov = wep['invoice_origin'].sort_values(ascending=False)
cov_name = cov.keys()[0]
cov_cant = cov.loc[cov_name]


In [ ]:
print(f'La vendedora {vend_nom} tiene a sus 3 mejores clientes de la siguiente manera:')
print(f'\tEl cliente {cov_name} es el cliente que más órdenes de venta tiene, siendo un total de {cov_cant:,.2f} órdenes')
print(f'\tEl cliente {cprod_name} es el cliente que más cantidad de producto, siendo un total de {cprod_cant:,.2f} artículos vendidos')
print(f'\tEl cliente {csub_name} es el cliente que más facturó, siendo un total de ${csub_cant:,.2f} pesos en el mes de enero')